Note that you need python 3.7 to have use datetime.datetime.fromisoformat()

In [ ]:
import sys
assert sys.version_info >= (3, 7)

# Load data

In [ ]:
import re
import csv
import itertools
import numpy as np

In [ ]:
def load_data(path, limit=None):
    with open(path, encoding='utf8') as file:
        reader = csv.reader(file, delimiter=',')
        # get header
        header = next(reader)
        data = [[value for value in row]
                for row in itertools.islice(reader, limit)]
    return np.asarray(header), np.asarray(data)

In [ ]:
header, data = load_data("data/training.csv", limit=10000)

# I/ Data preprocessing

## I.1. Remove features with mostly missing values

In [ ]:
def print_sample(header, data, n=0):
    for i, (feature, value) in enumerate(zip(header, data[n])):
        print("({:^2d}) {:30} : {}".format(i, feature, value))

In [ ]:
def print_feature(header, data, max_feature=5):
    for n_feature, feature in enumerate(data.T):
        values, counts = np.unique(feature, return_counts=True)
        counts_values = sorted(zip(counts, values), reverse=True)
        print("-" * 50)
        print("({:02d}) {} ({})".format(n_feature, header[n_feature],
                                        len(values)))
        print("-" * 50)
        for i, (v, c) in enumerate(counts_values):
            if i > max_feature:
                break
            print("{:10} : {:10} ({:5.1%})".format(c, v, v / data.shape[0]))

In [ ]:
print_feature(header, data, 3)

 * We can remove the feature without values in over 50% of samples. 

 * We decide to keep Weather as it is discrete and we can easily replace it with a one-hot vector. 

 * We also need to remove Withdrawals that is not available in test data.

In [ ]:
def delete_feature(header, data, feature_name):
    assert feature_name in header, "Index of {} does not exist".format(
        feature_name)

    index = np.where(header == feature_name)
    return np.delete(header, index), np.delete(data, index, 1)

In [ ]:
header, data = delete_feature(header, data, "Visility indicator")
header, data = delete_feature(header, data, "hmdx")
header, data = delete_feature(header, data, "Wind Chill")

In [ ]:
print(data.shape)
print(header)

## I.2 Convert Date to Year, Month, Day, Hour

In the date we can extract several informations : the Year, the Month, the day, and the hour.

From this, we can also deduce a useful information : the day of the week (if it's Monday, Tuesday, etc.).

In [ ]:
from datetime import datetime

In [ ]:
def convert_date(header, data):
    assert "Date/Hour" in header, "Index of Date/Hour does not exist"

    new_data = []
    index = np.where(header == "Date/Hour")

    for i, d in enumerate(data):
        dt = datetime.fromisoformat(d[index][0])
        new_data.append(
            np.concatenate(
                (np.delete(d,
                           index), [dt.year, dt.month, dt.day,
                                    dt.hour], np.eye(7)[dt.date().weekday()])))

    date_header = [
        "Year", "Month", "Day", "Hour", "Monday", "Tuesday", "Wednesday",
        "Thursday", "Friday", "Saturday", "Sunday"
    ]
    new_header = np.concatenate((np.delete(header, index), date_header))

    return np.asarray(new_header), np.asarray(new_data)

In [ ]:
header, data = convert_date(header, data)

In [ ]:
print(data.shape)
print(header)

### I.2.1 One Hot encoding for Year and Month

In [ ]:
def convert_one_hot(header, data, feature_name):
    assert feature_name in header, "Index of {} does not exist".format(
        feature_name)

    index = np.where(header == feature_name)
    
    new_header, new_data = [], []
    
    mapping, enc = np.unique(data[:, index], return_inverse=True)
    
    add_header = [feature_name + " " + str(m) for m in mapping]
    
    new_header = np.concatenate((np.delete(header, index), add_header))

    for i, (d, e) in enumerate(zip(data, enc)):
        v = np.eye(mapping.shape[0])[e]
        new_data.append(np.concatenate((np.delete(d, index), v)))

    return np.asarray(new_header), np.asarray(new_data)

In [ ]:
header, data = convert_one_hot(header, data, "Year")
header, data = convert_one_hot(header, data, "Month")

In [ ]:
print(data.shape)
print(header)

## I.3 Convert Weather to binary vector

In [ ]:
def convert_weather(header, data, weather):
    assert "Weather" in header, "Index of Weather does not exist"

    new_data = []
    N = len(weather)
    index = np.where(header == "Weather")

    for i, d in enumerate(data):
        new_weather = [
            1 if any([w == v for v in d[index][0].split(",")]) else 0
            for w in weather
        ]
        new_data.append(np.concatenate((np.delete(d, index), new_weather)))

    new_header = np.concatenate((np.delete(header, index), weather))

    return np.asarray(new_header), np.asarray(new_data)

In [ ]:
weather = [
    'Orages', 'Brouillard', 'Bruine', 'Généralement dégagé',
    'Généralement nuageux', 'Pluie', 'Pluie modérée', 'Pluie forte', 'Dégagé',
    'Nuageux', 'Neige'
]

header, data = convert_weather(header, data, weather)

In [ ]:
print(data.shape)
print(header)

## I.4. Convert feature type from string to float (remove samples with missing values)

In [ ]:
# samples with at least one missing value
missing = [d for d in data if "" in d]
print(len(missing))

# number of class 1 with missing value
index = np.where(header == "Volume")
print(sum(["1" in d[index] for d in missing]))

Let's remove the samples with missing values as only one hundred have label 1.

In [ ]:
def convert_type(data):
    return np.asarray([[float(v.replace(",", ".")) for v in d] for d in data
                       if "" not in d])

In [ ]:
data = convert_type(data)

In [ ]:
print(data.shape)
print(header)

## I.5. Normalization of continuous data

The concerned features are : Temperature, Drew point, Relativite humidity, wind direction, Wind speed, and Pressure at the station

In [ ]:
def normalization_feature(header, data, feature_name):
    assert feature_name in header, "Index of {} does not exist".format(
        feature_name)
    index = np.where(header == feature_name)
    data[:, index] = (data[:, index] - np.mean(data[:, index])) / np.std(
        data[:, index])

In [ ]:
normalization_feature(header, data, "Temperature (°C)")
normalization_feature(header, data, "Drew point (°C)")
normalization_feature(header, data, "Relativite humidity (%)")
normalization_feature(header, data, "wind direction (10s deg)")
normalization_feature(header, data, "Wind speed (km/h)")
normalization_feature(header, data, "Pressure at the station (kPa)")

## I.6. Get x, y (withdrawals) and label (volume)

In [ ]:
def split(header, data):
    y_index = np.where(header == "Withdrawals")
    l_index = np.where(header == "Volume")

    y = data[:, y_index].reshape(-1)
    label = data[:, l_index].reshape(-1)
    x = np.delete(data, (y_index, l_index), 1)

    header = np.delete(header, (y_index, l_index))
    
    return header, x, y, label

In [ ]:
header, x, y, label = split(header, data)

# II/ Data analysis & visualization

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

## II.1. Distribution of the features

In [ ]:
def plot_feature(header, x, feature_name):
    assert feature_name in header, "Index of {} does not exist".format(
        feature_name)
    index = np.where(header == feature_name)

    plt.figure(figsize=(6, 4), dpi=300)
    sns.distplot(x[:, index])
    plt.show()

In [ ]:
plot_feature(header, x, "Temperature (°C)")

## II.2. Correlation of features and output

### II.2.1. Correlation matrix of the features

In [ ]:
def corr_matrix(header, x):
    mask = np.zeros((x.shape[1], x.shape[1]), dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    plt.figure(figsize=(14, 12), dpi=300)
    sns.heatmap(np.corrcoef(x.T),
                mask=mask,
                center=0,
                cmap=cmap,
                square=True,
                linewidths=.5,
                cbar_kws={"shrink": .5},
                xticklabels=header,
                yticklabels=header)
    plt.show()

### II.2.2. Correlation between each features with the output

In [ ]:
def feature_output_corr(header, x, y, limit=None):
    coeff = [np.corrcoef(feature, y)[0][1] for feature in x.T]
    abs_coeff = list(map(abs, coeff))

    for _, coeff, name in itertools.islice(
            sorted(zip(abs_coeff, coeff, header), reverse=True), limit):
        print("{:30} : {:6.3f}".format(name, coeff))

In [ ]:
feature_output_corr(header, x, y, 10)

In [ ]:
feature_output_corr(header, x, label, 10)

# Pipeline

In [ ]:
def pipeline(path="data/training.csv",
             limit=None,
             delete_features=["Visility indicator", "hmdx", "Wind Chill"],
             cvrt_date=True,
             weather=[
                 "Orages", "Brouillard", "Bruine", "Généralement dégagé",
                 "Généralement nuageux", "Pluie", "Pluie modérée",
                 "Pluie forte", "Dégagé", "Nuageux", "Neige"
             ],
             one_hot_features=["Year", "Month"]):
    """
    path :           (STRING) path of the file to load.
    limit:           (INT) limit the number of example to load.
    delete_features: (LIST) feature names to remove.
    cvrt_date:       (BOOLEAN) convert the data
    weather:         (LIST) weather to consider. All other will be dropped.
    one_hot_features (LIST) feature names to convert in one-hot vector
    """
    header, data = load_data(path, limit)

    for f in delete_features:
        header, data = delete_feature(header, data, f)

    if cvrt_date:
        header, data = convert_date(header, data)

    for f in one_hot_features:
        header, data = convert_one_hot(header, data, f)

    if weather:
        header, data = convert_weather(header, data, weather)

    data = convert_type(data)

    return split(header, data)

# Models

## Load and split data

In [ ]:
header, x, y, label = pipeline(limit=1000)

In [ ]:
split = int(x.shape[0] * 0.8)
x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]
label_train, label_test = label[:split], label[split:]

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [ ]:
def compute_f1(proba, y_true, step=0.01, plot=False):
    f1 = []

    for threshold in np.arange(0, 1, step):
        y_pred = [int(y > threshold) for y in proba]
        f1.append(f1_score(y_true, y_pred) if 1 in y_pred else 0)

    if plot:
        plt.figure(figsize=(6, 4), dpi=300)
        plt.plot(np.arange(0, 1, step), f1)
        plt.xlabel('Threshold')
        plt.ylabel('F1-score')
        plt.show()

    return max(f1), step * np.argmax(f1)

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=9999, class_weight={0: 1, 1: 6})
model = model.fit(x_train, label_train)

In [ ]:
prediction = model.predict_proba(x_test)
proba = list(zip(*prediction))[1]

In [ ]:
compute_f1(proba, label_test, plot=True)